<img src="dsci512_header.png" width="600">

# Lecture 4 Sparse matrices

## Topics
- Dask and parallelized computation on CPUs 
- Pytorch and GPU computation
- Data types
- Sparse matrices
- Bag-of-word matrices

## Learning objectives

- Identify situations where matrix parallelization is necessary
- Understand the date types
- Understand sparse matrix representations
- Understand bag-of-word matrices

#### Block matrix operations

A block matrix is a matrix that is divided into smaller rectangular or square submatrices, which are called blocks.

When doing matrix multiplication, you can split a large matrix into smaller blocks and perform operations on these blocks independently. You can get the exact same result as doing multiplication with the whole matrix. 

In [ ]:
import numpy as np

# Create a large matrix A
A = np.array([[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12],
              [13, 14, 15, 16]])

# Partition A into four blocks
A11 = A[:2, :2]
A12 = A[:2, 2:]
A21 = A[2:, :2]
A22 = A[2:, 2:]

# Compute A^2 using block matrix operations
A_squared_block = np.block([[np.dot(A11, A11) + np.dot(A12, A21), np.dot(A11, A12) + np.dot(A12, A22)],
                            [np.dot(A21, A11) + np.dot(A22, A21), np.dot(A21, A12) + np.dot(A22, A22)]])

# Compute A^2 using standard matrix multiplication
A_squared = np.dot(A, A)

# Check if the two results are equivalent
equivalent = np.array_equal(A_squared, A_squared_block)

print("A^2 computed using block operations:\n", A_squared_block)
print("\nA^2 computed using standard matrix multiplication:\n", A_squared)
print("\nAre the results equivalent?", equivalent)


A^2 computed using block operations:
 [[ 90 100 110 120]
 [202 228 254 280]
 [314 356 398 440]
 [426 484 542 600]]

A^2 computed using standard matrix multiplication:
 [[ 90 100 110 120]
 [202 228 254 280]
 [314 356 398 440]
 [426 484 542 600]]

Are the results equivalent? True


We can utilize this nice property to speed up matrix computation. We can do operations on individual blocls means we can parallelize the operations, that is, distribute the task to individual processes. 

[Dask](https://www.dask.org/) is a flexible parallel computing library for Python that is designed to scale computations across multiple cores or even distributed systems.


To use dask, we first need to start the client. 

We need to Start the Client. This is just for setting up dask. You need to do it once at the start of your program and ignore it afterwards. 

In [1]:
import dask
from dask.distributed import Client
# set a temporary folder to store some intermediate results generated by dask.
dask.config.set(temporary_directory='../dask_tmp')
client = Client(n_workers=2) 
# set how many processes you want. this depends on how many cores your computer has. generally speaking, if the scale of your problem is large enough, more cores lead to better performance (assuming that your computation can be parallelized).
client # get client information

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 6,Total memory: 15.56 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40135,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 6
Started: Just now,Total memory: 15.56 GiB
Comm: tcp://127.0.0.1:37213,Total threads: 3
Dashboard: http://127.0.0.1:41589/status,Memory: 7.78 GiB
Nanny: tcp://127.0.0.1:42535,


Here is an example of numpy vs. dask. 

In [3]:
import numpy as np
import dask.array as da
import time

# Create a large NumPy array
numpy_array = np.random.rand(10**8)

# Create a large Dask array
# here we split the numpy array into chunks of 10**6
dask_array = da.from_array(numpy_array, chunks=10**6).persist()

# Perform a computationally intensive operation using NumPy
def complex_operation(arr):
    return np.sin(arr) + np.cos(arr) * np.tan(arr)

start_time = time.time()
result_numpy = complex_operation(numpy_array)
print("NumPy time:", time.time() - start_time)

# Perform the same operation using Dask with parallelism
# Dask will automatically manage the parallization on individual chunks for you
# its syntax is almost the same as numpy
def complex_operation_dask(arr):
    return da.sin(arr) + da.cos(arr) * da.tan(arr)

start_time = time.time()
result_dask = complex_operation_dask(dask_array)
result_dask = result_dask.compute()  # Compute the result in parallel. If you don't call compute(), dask will not run and nothing will be returned. This is called lazy evaluation. Remember `range(n)` in Python? 
print("Dask time:", time.time() - start_time)

# Compare the results
print(np.allclose(result_numpy, result_dask))


/home/lukeum/anaconda3/envs/py10/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 762.94 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


NumPy time: 5.186022996902466
Dask time: 3.5584545135498047
True


Let's choose a different chunk size. 

In [6]:
# Create a large NumPy array
numpy_array = np.random.rand(10**8)

# Create a large Dask array
dask_array = da.from_array(numpy_array, chunks=10**5).persist()

# Perform the same operation using Dask with parallelism
# Dask will automatically manage the parallization on individual chunks for you
# its syntax is almost the same as numpy
def complex_operation_dask(arr):
    return da.sin(arr) + da.cos(arr) * da.tan(arr)

start_time = time.time()
result_dask = complex_operation_dask(dask_array)
result_dask = result_dask.compute()  # Compute the result in parallel. If you don't call compute(), dask will not run and nothing will be returned. This is called lazy evaluation. Remember `range(n)` in Python? 
print("Dask time:", time.time() - start_time)

# Compare the results
print(np.allclose(result_numpy, result_dask))


/home/lukeum/anaconda3/envs/py10/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 762.96 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Dask time: 37.812100410461426
False


It's now taking a much longer time! 
Why? 

There is communication overhead associated with parallelized computation. If the chunksize is small, the processing time might be overwhelmed by the time for communication between different processes, bringing no speed-up at all!

Generally speaking, the chunks should be more than the number of cores you allocated (why? if you have 4 cores and split the matrix into 2 chunks, what will happen?).
The chunks should not be too small, such that the time needed to process each chunk is much more than the communication overhead. 

Paralleling matrices across CPUs can speed up the computation. We can get even much more speed-up by using GPUs!

### Massively parallelized matrix operations

PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing, originally developed by Meta AI and now part of the Linux Foundation umbrella. It is free and open-source software released under the modified BSD license.

Numpy arrays are mainly used in typical machine learning algorithms (such as k-means or Decision Tree in scikit-learn) whereas pytorch tensors are mainly used in deep learning which requires heavy matrix computation.

Example:

Cosine similarity measures the cosine of the angle between two non-zero vectors $a$ and $b$. It quantifies how similar the two vectors are, regardless of their magnitude (length). Because we normalize them to be length 1.

$$
sim(a,b) = \frac{a}{||a||}\cdot\frac{b}{||b||} = \frac{ab}{||a||\cdot||b||}
$$

The value of cosine similarity ranges from -1 to 1:
 - 1 means the vectors are identical (pointing in the same direction).
 - 0 means the vectors are orthogonal (perpendicular), indicating no similarity.
 - -1 means the vectors are diametrically opposed (pointing in opposite directions).


In [1]:
import numpy as np
import torch
import time

# Define the matrix sizes
size = 500000
num_iterations = 10

# NumPy Matrix Multiplication
a = np.random.rand(size)
b = np.random.rand(size)

start_time = time.time()
for _ in range(num_iterations):
    np_result = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
np_numpy_time = (time.time() - start_time) 
print(f"NumPy Runtime: {np_numpy_time:.4f} seconds")

NumPy Runtime: 0.0028 seconds


In [2]:

# PyTorch Matrix Multiplication
a = torch.rand(size)
b = torch.rand(size)

start_time = time.time()
for _ in range(num_iterations):
    torch_result = torch.dot(a, b)/(torch.norm(a)*torch.norm(b))
torch_pytorch_time = (time.time() - start_time)


print(f"PyTorch Runtime: {torch_pytorch_time:.4f} seconds")


PyTorch Runtime: 0.0874 seconds


Pytorch isn't that fast on CPU. However, it is optimized for GPU computation. 

Tips: if you don't have a GPU on your computer, you can open this notebook on [Google Colab](https://colab.research.google.com/) and select T4 as your runtime. Then you can run you jobs on GPU. 

In [4]:
device = 'cuda'
# PyTorch Matrix Multiplication
a = torch.rand(size).to(device)
b = torch.rand(size).to(device)

start_time = time.time()
for _ in range(num_iterations):
    torch_result = torch.dot(a, b)/(torch.norm(a)*torch.norm(b))
torch_pytorch_time = (time.time() - start_time) 

print(f"PyTorch on GPU Runtime: {torch_pytorch_time:.4f} seconds")


PyTorch on GPU Runtime: 0.0014 seconds


What is a GPU? 

A GPU, or Graphics Processing Unit, is a special type of computer chip originally designed to handle complex tasks related to graphics and images. However, GPU's ability to handle complex matrix operations makes it possible to train neural networks like [AlexNet](https://en.wikipedia.org/wiki/AlexNet). Hence begins the neural network revolution that you and I are currently experiencing. 

PyTorch's significant speed advantage when using GPUs compared to CPUs is due to several key factors:

- **Parallelism**: Modern GPUs are designed with thousands of cores, which allow them to perform many operations in parallel. PyTorch is capable of harnessing this parallelism, making it highly efficient for tasks like deep learning where many matrix multiplications occur simultaneously.

- **Optimized Libraries**: PyTorch leverages highly optimized GPU libraries, such as NVIDIA cuBLAS and cuDNN, to accelerate mathematical operations. These libraries are finely tuned for specific GPU architectures and provide substantial speed improvements over CPU-based implementations.

- **Data Transfer**: When using PyTorch on a GPU, data transfer between the CPU and GPU is typically minimized. This is achieved by loading data directly onto the GPU and keeping it there as much as possible, avoiding costly data transfers that can occur when using CPUs.


It's important to note that not all tasks benefit equally from GPU acceleration. Simple, small-scale operations may not see a substantial speed improvement on GPUs compared to CPUs. However, for computationally intensive tasks such as deep learning, scientific simulations, and large-scale linear algebra operations, GPUs can provide orders of magnitude in performance improvements.

Pytorch has similar operations in Numpy, but with a slightly different syntax. For example, an array in numpy is called tensor in pytorch. Below are some simple examples. 

In [ ]:
# Create a tensor
x = torch.Tensor(2, 3, 4)
print(x)

# Create a tensor from a (nested) list
x = torch.Tensor([[1, 2], [3, 4]])
print(x)

# Create a tensor with random values between 0 and 1 with the shape [2, 3, 4]
x = torch.rand(2, 3, 4)
print(x)

In [ ]:
shape = x.shape
print("Shape:", x.shape)

size = x.size()
print("Size:", size)

dim1, dim2, dim3 = x.size()
print("Size:", dim1, dim2, dim3)

In [ ]:
x = torch.arange(12).view(3, 4)
print("X", x)

print(x[:, 1])   # Second column
print(x[0])      # First row
print(x[:2, -1]) # First two rows, last column
print(x[1:3, :]) # Middle two rows

In [ ]:
x = torch.arange(6)
x = x.view(2, 3) # view is reshape in numpy
print("X", x)

In [ ]:
W = torch.arange(9).view(3, 3) # We can also stack multiple operations in a single line. 
print("W", W)

In [ ]:
h = torch.matmul(x, W) # Verify the result by calculating it by hand too!
print("h", h)

### Speeding up matrix multiplication with half-precision

Note. A GPU is needed to run this experiment.  

In computer engineering, decimal numbers like 1.0151 or 566132.8 are traditionally represented as floating point numbers. Since we can have infinitely precise numbers (think π), but limited space in which to store them, we have to make a compromise between precision (the number of decimals we can include in a number before we have to start rounding it) and size (how many bits we use to store the number).

The technical standard for floating point numbers, IEEE 754 (for a deep dive I recommend the PyCon 2019 talk Floats are Friends: making the most of IEEE754.00000000000000002), sets the following standards:

 - `fp64`, aka double-precision or “double”, max rounding error of ~2^-52

 - `fp32`, aka single-precision or “single”, max rounding error of ~2^-23

 - `fp16`, aka half-precision or “half”, max rounding error of ~2^-10

Python uses fp64 for the float type. PyTorch, which is much more memory-sensitive, uses fp32 as its default dtype instead.

If we halve the precision (fp32 → fp16), we halve the time and space at the cost of precision (only if this is supported by specific hardware, i.e. GPU).

It's rare that we need to do highly accurate matrix computations at scale. In fact, often we're doing some kind of machine learning, and less accurate approaches can prevent overfitting.

If we accept some decrease in accuracy, then we can often increase speed by orders of magnitude (and/or decrease memory use).

Let's look at an example with cosine similarity. 

In [5]:
# Define the matrix sizes
size = 500000
num_iterations = 10000

# NumPy Matrix Multiplication
a = np.random.rand(size)
b = np.random.rand(size)
print(a.dtype)

start_time = time.time()
for _ in range(num_iterations):
    np_result = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b)) # this line is vectorized
np_numpy_time = (time.time() - start_time) 
print(f"NumPy Runtime: {np_numpy_time:.4f} seconds")

float64
NumPy Runtime: 2.5368 seconds


In [6]:
# NumPy Matrix Multiplication
a = np.random.rand(size).astype(np.float32)
b = np.random.rand(size).astype(np.float32)

print(a.dtype)

start_time = time.time()
for _ in range(num_iterations):
    np_result = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
np_numpy_time = (time.time() - start_time) 
print(f"NumPy Runtime: {np_numpy_time:.4f} seconds")

float32
NumPy Runtime: 1.5467 seconds


In [ ]:
# NumPy Matrix Multiplication
a = np.random.rand(size).astype(np.float16)
b = np.random.rand(size).astype(np.float16)

start_time = time.time()
for _ in range(num_iterations):
    np_result = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
np_numpy_time = (time.time() - start_time) 
print(f"NumPy Runtime: {np_numpy_time:.4f} seconds")

This example above probably gives you a numerical error. Because low precision computation can easily lead to underflow or overflow, triggering errors. 

 - Overflow: When numbers get too big for the computer to handle, they wrap around or behave unexpectedly.
 - Underflow: When numbers get too small, they are rounded down to zero, losing important information.

Let's look at how  `pytorch` does it.

In [7]:
# Set the size of the vectors to be created
size = 500000  

# Set the number of iterations 
num_iterations = 10000  

# Specify the device for computation (GPU). You need this to enable GPU computation.
device = 'cuda' 

# Create two random vectors (arrays) using PyTorch
a = torch.rand(size).to(device)  
b = torch.rand(size).to(device)  


print(a.dtype)  

# Start measuring the time for the dot product computation
start_time = time.time()  


for _ in range(num_iterations):  
    # Calculate the cosine similarity between vectors 'a' and 'b'
    torch_result = torch.dot(a, b) / (torch.norm(a) * torch.norm(b))  


torch_pytorch_time = (time.time() - start_time)


print(f"PyTorch Runtime: {torch_pytorch_time:.4f} seconds")  

torch.float32
PyTorch Runtime: 0.6180 seconds


In [9]:

# PyTorch Matrix Multiplication
a = torch.rand(size).half().to(device)
b = torch.rand(size).half().to(device)
print(a.dtype)

start_time = time.time()
for _ in range(num_iterations):
    torch_result = torch.dot(a, b)/(torch.norm(a)*torch.norm(b))
torch_pytorch_time = (time.time() - start_time)


print(f"PyTorch Runtime: {torch_pytorch_time:.4f} seconds")


torch.float16
PyTorch Runtime: 0.6301 seconds


Using `float16`, aslo known as half precision, can reduce the computation time/memory by half! You might not see the speed up in this example. We need a problem of bigger scale to see the actual improvement. If you increase the size or the iterations by 100 times, you should see the speed up.
Of course, a price we must pay is the reduced precision, which makes numerical underflow and overflow more frequent.

## Intro to sparse matrices

In [11]:
import scipy.sparse
import networkx as nx

Sparse matrices are a class of matrices with most of its elements being 0. 
Let's initialize a sparse matrix.  
(Here we are initializing the graph adjcency matrix, which we will cover in week 3 on graph data structure. Here you just need to know that it is a matrix with a lot of zeros in it.  )

In [ ]:
G = nx.ladder_graph(5)
# Return the Ladder graph of length n.
# This is two rows of n nodes, with each pair connected by a single edge.
am_ladder = nx.adjacency_matrix(G)

In [ ]:
type(am_ladder) # Compressed Sparse Row

scipy.sparse._csr.csr_matrix

- Sparse matrices are a conceptual data structure like a list, dictionary, set, etc.
- [`scipy.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html) matrices are the standard Python implementation of this conceptual data structure, like `list`, `dict`, `set`, etc.
- Going to that link, we can see there are many types of scipy sparse matrix.
  - This one is a [`csr_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix)
  - More later on these types.
- You can convert them to numpy arrays with `toarray()`, but this is often a bad idea.

In [ ]:
print(am_ladder.toarray())

[[0 1 0 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 1 0 0 0]
 [0 1 0 1 0 0 0 1 0 0]
 [0 0 1 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 1 0 1 0 0]
 [0 0 1 0 0 0 1 0 1 0]
 [0 0 0 1 0 0 0 1 0 1]
 [0 0 0 0 1 0 0 0 1 0]]


In [ ]:
A = am_ladder.toarray() # undirect graph = symmetric matrix;

print(A)
print("---")
print(A.T)

[[0 1 0 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 1 0 0 0]
 [0 1 0 1 0 0 0 1 0 0]
 [0 0 1 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 1 0 1 0 0]
 [0 0 1 0 0 0 1 0 1 0]
 [0 0 0 1 0 0 0 1 0 1]
 [0 0 0 0 1 0 0 0 1 0]]
---
[[0 1 0 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 1 0 0 0]
 [0 1 0 1 0 0 0 1 0 0]
 [0 0 1 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 1 0 1 0 0]
 [0 0 1 0 0 0 1 0 1 0]
 [0 0 0 1 0 0 0 1 0 1]
 [0 0 0 0 1 0 0 0 1 0]]


In [12]:
# make a bigger sparse matrix
G = nx.fast_gnp_random_graph(100_000,1e-4) 
am = nx.adjacency_matrix(G)
type(am)

scipy.sparse._csr.csr_array

In [ ]:
am.shape # 10 billion

(100000, 100000)

In [ ]:
am.nnz # non-zero elements of the matrix 

999120

Stored in full form, the matrix would take up:

In [ ]:
import numpy as np

In [ ]:
%timeit np.sum(am, axis = 0) # row-wise sum

3.2 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# https://numpy.org/doc/stable/reference/generated/numpy.sum.html
print(np.sum([[0, 1], [2, 5]], axis=None))  # default

print(np.sum([[0, 1], [2, 5]], axis=1))     # row-wise sum
print(np.sum([[0, 1], [2, 5]], axis=0))     # column-wise sum

print(np.sum([[0, 1], [2, 5]], axis=-1))     # last axis ->  row
print(np.sum([[0, 1], [2, 5]], axis=-2))     # second last axis ->  column

8
[1 7]
[2 6]
[1 7]
[2 6]


In [ ]:
import numpy as np

full_size = int(np.prod(am.shape))*8/(1e9)
print("The full matrix would take up %d GB" % full_size)

The full matrix would take up 80 GB


In [ ]:
x = am.toarray()

print("Size of the array: ",
      x.size)
 
print("Memory size of one array element in bytes: ",
      x.itemsize)

print("Memory size of numpy array in bytes:",
      x.size * x.itemsize, "Bytes")

print("Memory size of numpy array in gigabytes:",
      (x.size * x.itemsize) / (1e9), "GB")                        # 1 byte = 1e-9 gigabyte

Size of the array:  10000000000
Memory size of one array element in bytes:  8
Memory size of numpy array in bytes: 80000000000 Bytes
Memory size of numpy array in gigabytes: 80.0 GB


That's a lot! How big is the sparse matrix?

In [ ]:
sparse_size = am.data.nbytes/1e6
print("The sparse matrix takes up %d NB" % sparse_size)

The sparse matrix takes up 7 NB


So, the fraction of space saved is:

In [ ]:
frac_nz = am.nnz / np.prod(am.shape)
print("The sparse matrix is %dx smaller" % (1/frac_nz))

The sparse matrix is 10009x smaller


- Right, so we definitely don't want to store the full matrix.

- Regular numpy functions work on sparse matrices, although they might be fast/slow depending on various factors. 
- You definitely do not want to iterate through the rows - make sure you use built-in numpy functions. 

#### Sparse datasets

Sparse matrices come up _a lot_ in practice. For example:

- Word counts: we might represent a document by which words are in it, but only a small fraction of all words would appear in a given document.
- Ratings: we might represent an Amazon item by the user ratings, but only a small fraction of all users have rated a given item.
- Physical processes: in a 2019 Capstone project, MDS students examined images from a particle physics dataset, in which most of the sensors got zero signal.
- etc.

## Building sparse matrices

To create an empty sparse martrix, just provide the shape as the argument to the constructor

In [ ]:
from scipy.sparse import csr_matrix,csc_matrix,lil_matrix
# csr: Compressed Sparse Row array
# csc: Compressed Sparse Column array
# lil: Row-based list of lists sparse array

shape = (10,10)
#my code here
x = csr_matrix(shape)
#my code here
x.shape

(10, 10)

 However, this is a bad way to build a sparse matrix of any significant size. In fact, scipy will warn you that it's a very bad idea to try to populate a CSR sparse matrix directly

In [ ]:
x[5,5] = 1

/Users/jungyeul/Library/Python/3.9/lib/python/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


A better way? Build the matrix in another sparse format, and then convert. Let build a diagonal matrix using the lil (linked list, i.e. the treasure boxes) format, which doesn't support efficient row or column operations like the csr and csc matrices, but does support efficient assignment.

In [ ]:
#my code here
shape = (10,10)

x = lil_matrix(shape)
for i in range(shape[0]):
    x[i,i] = 1
    
x = csr_matrix(x)
print(x)
#my code here

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	1.0
  (7, 7)	1.0
  (8, 8)	1.0
  (9, 9)	1.0


The Scikit learn package allows you to create csr sparse matrices from Python dictionaries (using [DictVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html)) and even directly from raw text strings (using [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html);).



In [ ]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)
D1 = [{'foo': 1, 'bar': 2}, {'foo': 3, 'baz': 1}]
X1 = dv.fit_transform(D1)
print(dv.get_feature_names_out())
print(X1)
print(type(X1))
# print(dv.vocabulary_)
print("----")


['bar' 'baz' 'foo']
[[2. 0. 1.]
 [0. 1. 3.]]
<class 'numpy.ndarray'>
----


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
D2 = ['foo bar baz', 'bar foo', 'foo', 'bar', 'bar']
X2 = cv.fit_transform(D2)
print(cv.get_feature_names_out())
print(X2)
print(type(X2))
print(X2.toarray())
# print(cv.vocabulary_)

['bar' 'baz' 'foo']
  (0, 2)	1
  (0, 0)	1
  (0, 1)	1
  (1, 2)	1
  (1, 0)	1
  (2, 2)	1
  (3, 0)	1
  (4, 0)	1
<class 'scipy.sparse._csr.csr_matrix'>
[[1 1 1]
 [1 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]]


Optional example

In [9]:
# #pip install nltk
import nltk
nltk.download("brown")

[nltk_data] Downloading package brown to /home/lukeum/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [13]:
from nltk.corpus import brown

Then, we apply the vectorizer to convert this to a csr matrix. First, initialize the vectorizer, then use its `fit_transform` method

`CountVectorizer()`

Convert a collection of text documents to a matrix of token counts.

This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer()
count_vec_output = count_vec.fit_transform(brown.words('ca01')) 

# print the identified unique words along with their indices
print(count_vec.vocabulary_['fulton'])

284


In [17]:
#print(count_vec_output)
print(type(count_vec_output))
print(count_vec_output.toarray())

<class 'scipy.sparse._csr.csr_matrix'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Pitfalls 

#### Matrix vs. Array 

- Note that when you do an operation which on sparse matrix that removes one of the dimensions (as we did above), you actually don't have a sparse matrix anymore. 
- Surprisingly, you also don't have a numpy array! 
- What you have a numpy [matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html), which is distinct from an array. 
    - See [here](https://numpy.org/devdocs/user/numpy-for-matlab-users.html#array-or-matrix-which-should-i-use) for a discussion of the differences between arrays and matrices. 
- Rule of thumb: (non-sparse) matrices will cause you headaches and you should stay away from them. 
- You can access an array version of the matrix (without copying it) using `np.asarray`, and you should probably do that right away

In [ ]:
type(wpt_matrix)

numpy.matrix

In [ ]:
wpt_array = np.asarray(wpt_matrix)
wpt_array[:10]

array([[2242.],
       [2277.],
       [2275.],
       [2217.],
       [2244.],
       [2263.],
       [2270.],
       [2187.],
       [2234.],
       [2282.]])

An example of where a numpy matrix will cause you trouble: operations like `flatten` (which is suppose to reduce a dataset to 1 dimension) won't work properly with a matrix, which is *always* 2-d

In [ ]:
wpt_matrix.flatten()

matrix([[2242., 2277., 2275., 2217., 2244., 2263., 2270., 2187., 2234.,
         2282., 2259., 2338., 2241., 2329., 2314., 2374., 2293., 2314.,
         2260., 2317., 2245., 2309., 2327., 2252., 2327., 2278., 2288.,
         2220., 2349., 2274., 2311., 2323., 2284., 2235., 2217., 2229.,
         2324., 2221., 2452., 2340., 2313., 2338., 2291., 2277., 2200.,
         2234., 2244., 2230., 2241., 2231., 2239., 2395., 2312., 2298.,
         2293., 2296., 2267., 2264., 2311., 2260., 2288., 2194., 2287.,
         2299., 2278., 2294., 2357., 2346., 2368., 2316., 2262., 2415.,
         2320., 2356., 2342., 2398., 2433., 2421., 2351., 2345., 2461.,
         2480., 2342., 2282., 2574., 2503., 2311., 2370., 2213., 2334.,
         2332., 2318., 2410., 2317., 2359., 2332., 2314., 2238., 2304.,
         2307., 2236., 2276., 2315., 2451., 2343., 2258., 2279., 2257.,
         2295., 2271., 2222., 2370., 2226., 2476., 2402., 2333., 2288.,
         2261., 2346., 2333., 2223., 2244., 2302., 2260., 2251.,

But this will work fine with the array version

In [ ]:
wpt_array.flatten()

array([2242., 2277., 2275., 2217., 2244., 2263., 2270., 2187., 2234.,
       2282., 2259., 2338., 2241., 2329., 2314., 2374., 2293., 2314.,
       2260., 2317., 2245., 2309., 2327., 2252., 2327., 2278., 2288.,
       2220., 2349., 2274., 2311., 2323., 2284., 2235., 2217., 2229.,
       2324., 2221., 2452., 2340., 2313., 2338., 2291., 2277., 2200.,
       2234., 2244., 2230., 2241., 2231., 2239., 2395., 2312., 2298.,
       2293., 2296., 2267., 2264., 2311., 2260., 2288., 2194., 2287.,
       2299., 2278., 2294., 2357., 2346., 2368., 2316., 2262., 2415.,
       2320., 2356., 2342., 2398., 2433., 2421., 2351., 2345., 2461.,
       2480., 2342., 2282., 2574., 2503., 2311., 2370., 2213., 2334.,
       2332., 2318., 2410., 2317., 2359., 2332., 2314., 2238., 2304.,
       2307., 2236., 2276., 2315., 2451., 2343., 2258., 2279., 2257.,
       2295., 2271., 2222., 2370., 2226., 2476., 2402., 2333., 2288.,
       2261., 2346., 2333., 2223., 2244., 2302., 2260., 2251., 2239.,
       2402., 2307.,

#### Indexing

Another issue with sparse matrices is indexing. With a regular numpy array, `x[i,j]` and `x[i][j]` are equivalent:

In [ ]:
x = np.random.rand(10,10)

In [ ]:
x[1,2]

0.5267092734326778

In [ ]:
x[1][2]

0.5267092734326778

This is because `x[1]` returns the first row, and then the `[2]` indexes into that row:

In [ ]:
x[1]

array([0.92771172, 0.76413423, 0.52670927, 0.24739091, 0.32841451,
       0.75422245, 0.1076381 , 0.28458965, 0.94934966, 0.90127702])

In [ ]:
row_1 = x[1]
row_1[2]

0.5267092734326778

However, with `scipy.sparse` matrices, things are a bit different:

In [ ]:
x_sparse = csr_matrix(x)

In [ ]:
x_sparse[1,2]

0.5267092734326778

In [ ]:
x_sparse[1][2]

IndexError: row index (2) out of range

Why?

In [ ]:
row_1_sparse = x_sparse[1]

In [ ]:
row_1_sparse.shape


(1, 10)

- The sparse matrix returns a different shape.
- This is because sparse matrices must always be 2d (same problem as non-sparse matrices)
- In general, use the `x[1,2]` notation when possible because chaining the `[]` can be problematic in several places (e.g., also pandas).
- However, **this is only for numpy**, not, say, a list of lists:

In [ ]:
lst = [[1,2,3],[4,5,6],[7,9]]

In [ ]:
lst[0][1]

2

In [ ]:
lst[0,1]

TypeError: list indices must be integers or slices, not tuple

#### Rows versus columns operations

Generally, you should avoid looping directly through sparse matrices.

But if you do need to, you need to be very sensitive to the difference between Row (CSR) and Column (CSC) sparse matricies. 

Note. CSR (Compressed Sparse Row) and CSC (Compressed Sparse Column) are two common formats for storing sparse matrices. Here’s a breakdown of the differences:

| Feature               | CSR                          | CSC                          |
|----------------------|------------------------------|------------------------------|
| Storage Arrays       | Values, Column Indices, Row Pointers | Values, Row Indices, Column Pointers |
| Optimized For        | Row operations               | Column operations             |
| Access Pattern       | Fast row slicing             | Fast column slicing           |
| Use Cases            | Matrix-vector multiplication, row-wise operations | Column-wise computations, matrix factorization |

You should not use CSR matrices for operations which require looping over columns. Observe:

In [ ]:
num_rows, num_cols = am.shape

In [ ]:
type(am)

scipy.sparse._csr.csr_matrix

In [ ]:
%%timeit 
(np.sum(am, axis = 0))

2.25 ms ± 39.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit 
(np.sum(am, axis = 1))

3.22 ms ± 14.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit -r1 -n1 

# iterate through the rows - this is much slower than numpy functions
for i in range(num_rows):
    am[i,:]

3.92 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
# %%timeit -r1 -n1

# # iterate through the columns - this is much MUCH slower
# for i in range(num_cols):
#     am[:,i]

6min 17s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Above: 

- Looping is slower, as per usual.
- But at least looping through the rows of a `csr_matrix` isn't that bad.
- However, looping through the columns of a `csr_matrix` is a disaster - it took several min on my laptop!!
  - Because it is stored _row by row_. 
  - To grab a single column, it needs to loop through each row and look for items in that column.
- If you want to loop through columns you should first convert the matrix type.

In [ ]:
am_csc = am.tocsc()

In [ ]:
%%timeit -r1 -n1

for i in range(num_cols):
    am_csc[:,i]

3.98 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Optional: Einstein Summation (einsum)

The Einstein summation convention is the ultimate generalization of products such as matrix multiplication to multiple dimensions. It offers a compact and elegant way of specifying almost any product of scalars/vectors/matrices/tensors. Despite its generality, it can reduce the number of errors made by computer scientists and reduce the time they spend reasoning about linear algebra. It does so by being simultaneously clearer, more explicit, more self-documenting, more declarative in style and less cognitively burdensome to use. Its advantages over such things as matrix multiplication are that it liberates its user from having to think about:

- The correct order in which to supply the argument tensors
- The correct transpositions to apply to the argument tensors
- nsuring that the correct tensor dimensions are lined up with one another
- The correct transposition to apply to the resulting tensor


The only things it requires are knowledge of:

- Along which dimensions to compute (inner/element-wise/outer) products.
- The desired output shape.


[EINSUM IS ALL YOU NEED - EINSTEIN SUMMATION IN DEEP LEARNING](https://rockt.github.io/2018/04/30/einsum)